In [1]:
import sympy
from sympy import sin

sympy.init_printing()  # for pretty printing

In [2]:
syms = sympy.symbols("theta phi")
theta, phi = syms
R = sympy.symbols("R")
m = sympy.Array(sympy.diag(R**2, R**2 * sin(theta)**2))
m


⎡ 2            ⎤
⎢R       0     ⎥
⎢              ⎥
⎢     2    2   ⎥
⎣0   R ⋅sin (θ)⎦

In [3]:
m_inv = m.applyfunc(lambda x: 0 if x == 0 else 1/x)
m_inv

⎡1             ⎤
⎢──      0     ⎥
⎢ 2            ⎥
⎢R             ⎥
⎢              ⎥
⎢        1     ⎥
⎢0   ──────────⎥
⎢     2    2   ⎥
⎣    R ⋅sin (θ)⎦

Compute Christoffel symbols:

In [4]:
dim = m.rank()
ch = sympy.MutableDenseNDimArray(range(dim*dim*dim), (dim,dim,dim))*0
ch[:,:,:]

⎡⎡0  0⎤  ⎡0  0⎤⎤
⎢⎢    ⎥  ⎢    ⎥⎥
⎣⎣0  0⎦  ⎣0  0⎦⎦

First prepare metric differential:

In [24]:
diff = sympy.Array([m.diff(s).tolist() for s in syms],(dim,dim,dim))
diff



⎡⎡0          0         ⎤        ⎤
⎢⎢                     ⎥  ⎡0  0⎤⎥
⎢⎢      2              ⎥  ⎢    ⎥⎥
⎣⎣0  2⋅R ⋅sin(θ)⋅cos(θ)⎦  ⎣0  0⎦⎦

Then compute [**Christoffel symbols**](https://hirata10.github.io/ph6820/lec08_christoffel.pdf) (section VI) with all three down indices:

In [6]:
for i in range(ch.shape[0]):
  for j in range(ch.shape[1]):
    for k in range(ch.shape[2]):
      ch[i,j,k] = 0.5 * (diff[k,j,i] + diff[j,i,k] - diff[i,k,j])
ch

⎡⎡0            0          ⎤  ⎡                           2              ⎤⎤
⎢⎢                        ⎥  ⎢         0            1.0⋅R ⋅sin(θ)⋅cos(θ)⎥⎥
⎢⎢         2              ⎥  ⎢                                          ⎥⎥
⎢⎣0  -1.0⋅R ⋅sin(θ)⋅cos(θ)⎦  ⎢     2                                    ⎥⎥
⎣                            ⎣1.0⋅R ⋅sin(θ)⋅cos(θ)           0          ⎦⎦

Raise first index via metric multiplication (tensorproduct followed by contration on the selected indices - one up from metric, first down from Christoffel symbol-):

In [7]:
temp = sympy.tensorproduct(m_inv, ch)
ch = temp[:,0,0,:,:] + temp[:,1,1,:,:]
ch

⎡                         ⎡            1.0⋅cos(θ)⎤⎤
⎢                         ⎢    0       ──────────⎥⎥
⎢⎡0          0         ⎤  ⎢              sin(θ)  ⎥⎥
⎢⎢                     ⎥  ⎢                      ⎥⎥
⎢⎣0  -1.0⋅sin(θ)⋅cos(θ)⎦  ⎢1.0⋅cos(θ)            ⎥⎥
⎢                         ⎢──────────      0     ⎥⎥
⎣                         ⎣  sin(θ)              ⎦⎦

Now for the Rieman Curvature Tensor. First prepare the Christoffel differential:

In [25]:
diff_ch = sympy.Array([ch.diff(s).tolist() for s in syms],(dim,dim,dim,dim))
diff_ch

⎡                                ⎡                                  2   ⎤⎤
⎢                                ⎢                           1.0⋅cos (θ)⎥⎥
⎢                                ⎢        0           -1.0 - ───────────⎥⎥
⎢⎡0              0            ⎤  ⎢                                2     ⎥⎥
⎢⎢                            ⎥  ⎢                             sin (θ)  ⎥⎥
⎢⎢          2             2   ⎥  ⎢                                      ⎥⎥
⎢⎣0  1.0⋅sin (θ) - 1.0⋅cos (θ)⎦  ⎢              2                       ⎥⎥
⎢                                ⎢       1.0⋅cos (θ)                    ⎥⎥
⎢                                ⎢-1.0 - ───────────          0         ⎥⎥
⎢                                ⎢            2                         ⎥⎥
⎢                                ⎣         sin (θ)                      ⎦⎥
⎢                                                                        ⎥
⎢            ⎡0  0⎤                               ⎡0  0⎤                 ⎥
⎢            ⎢    ⎥      

In [9]:
rie = sympy.MutableDenseNDimArray(range(dim*dim*dim*dim), (dim,dim,dim,dim))*0
rie

⎡⎡0  0⎤  ⎡0  0⎤⎤
⎢⎢    ⎥  ⎢    ⎥⎥
⎢⎣0  0⎦  ⎣0  0⎦⎥
⎢              ⎥
⎢⎡0  0⎤  ⎡0  0⎤⎥
⎢⎢    ⎥  ⎢    ⎥⎥
⎣⎣0  0⎦  ⎣0  0⎦⎦

Computing the **[Rieman](https://hirata10.github.io/ph6820/lec10_curvature.pdf) curvature tensor** (section III):

In [10]:
for i in range(rie.shape[0]):
  for j in range(rie.shape[1]):
    for k in range(rie.shape[2]):
      for l in range(rie.shape[3]):
        sum_1 = sum([ch[i,m,k]*ch[m,j,l] for m in range(dim)])
        sum_2 = sum([ch[i,m,l]*ch[m,j,k] for m in range(dim)])
        rie[i,j,k,l] = sympy.simplify(diff_ch[k,i,j,l] - diff_ch[l,i,j,k] + sum_1 - sum_2)
rie

⎡             ⎡                     2   ⎤⎤
⎢  ⎡0  0⎤     ⎢     0        1.0⋅sin (θ)⎥⎥
⎢  ⎢    ⎥     ⎢                         ⎥⎥
⎢  ⎣0  0⎦     ⎢        2                ⎥⎥
⎢             ⎣-1.0⋅sin (θ)       0     ⎦⎥
⎢                                        ⎥
⎢⎡ 0   -1.0⎤            ⎡0  0⎤           ⎥
⎢⎢         ⎥            ⎢    ⎥           ⎥
⎣⎣1.0   0  ⎦            ⎣0  0⎦           ⎦

Now compute the **Ricci tensor** by contracting the Rieman tensor:

In [11]:
ric = rie[0,:,0,:] + rie[1,:,1,:]
ric

⎡1.0       0     ⎤
⎢                ⎥
⎢            2   ⎥
⎣ 0   1.0⋅sin (θ)⎦

Note that due to symmetries, we could have chosen different down indices and get the analogous result:

In [12]:
rie[0,:,:,0] + rie[1,:,:,1]

⎡-1.0       0      ⎤
⎢                  ⎥
⎢              2   ⎥
⎣ 0    -1.0⋅sin (θ)⎦

and the other option would be all zeros:

In [13]:
rie[0,0,:,:] + rie[1,1,:,:]

⎡0  0⎤
⎢    ⎥
⎣0  0⎦

The Ricci tensor now has two down indices. Now compute the Ricci scalar, by first raising one index in the Ricci tensor, (via metric multiplication: tensor product followed by contraction):

In [14]:
temp = sympy.tensorproduct(m_inv,ric)
temp

⎡⎡1.0             ⎤                   ⎤
⎢⎢───       0     ⎥                   ⎥
⎢⎢  2             ⎥                   ⎥
⎢⎢ R              ⎥                   ⎥
⎢⎢                ⎥       ⎡0  0⎤      ⎥
⎢⎢            2   ⎥       ⎢    ⎥      ⎥
⎢⎢     1.0⋅sin (θ)⎥       ⎣0  0⎦      ⎥
⎢⎢ 0   ───────────⎥                   ⎥
⎢⎢           2    ⎥                   ⎥
⎢⎣          R     ⎦                   ⎥
⎢                                     ⎥
⎢                    ⎡   1.0         ⎤⎥
⎢                    ⎢──────────   0 ⎥⎥
⎢                    ⎢ 2    2        ⎥⎥
⎢      ⎡0  0⎤        ⎢R ⋅sin (θ)     ⎥⎥
⎢      ⎢    ⎥        ⎢               ⎥⎥
⎢      ⎣0  0⎦        ⎢            1.0⎥⎥
⎢                    ⎢    0       ───⎥⎥
⎢                    ⎢              2⎥⎥
⎣                    ⎣             R ⎦⎦

In [15]:
ric2 = temp[0,:,0,:] + temp[1,:,1,:]
ric2

⎡1.0     ⎤
⎢───   0 ⎥
⎢  2     ⎥
⎢ R      ⎥
⎢        ⎥
⎢     1.0⎥
⎢ 0   ───⎥
⎢       2⎥
⎣      R ⎦

And finally contract the Ricci tensor to get the **Ricci scalar**:

In [16]:
ric_s = ric2[0,0] + ric2[1,1]
ric_s

[**Einstein Tensor**](https://hirata10.github.io/ph6820/lec11_efe.pdf) (section IV.A, equation 15), with down indices, is then:

In [17]:
ein = ric - 0.5 * ric_s * m
ein

⎡0  0⎤
⎢    ⎥
⎣0  0⎦

which is 0 as expected for spherical polar coordinates, given that they correspond to flat spacetime.